In [1]:
! nvidia-smi

Tue Aug 27 10:23:08 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:25:00.0 Off |                    0 |
| N/A   36C    P0             66W /  300W |   17215MiB /  81920MiB |    100%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# pseudocode for current project baseline:
Initialise policy
Initialise Test_set, Training_set
Initialise training_argments
# prepare data for orpo fine-tune
For each sample in Training_set do:
    sample.Format_data(policy)
end For

Run orpo_fine-tuning with base_model, training_argments
Until epoch >= training_argments.num_of_epoch or validation_loss_difference <= training_argments.threshold

# evaluation both base model and fine-tuned model
For model in [base_model, fine-tuned_model] do:
    For prompt in Test_set do:
        if use_RAG:
            rag_result <- search(prompt, Training_set)
            response <- model.inference_with_RAG(policy, prompt, rag_result)
        else:
            response <- model.inference(policy, prompt)
    end For
    response.evluate()
end For

# pseudocode for proposed GAN project baseline:
Initialise policy
Initialise Test_set, Training_set
Initialise training_argments
# prepare data for orpo fine-tune
For each sample in Training_set do:
    sample.Format_data(policy)
end For

For epoch in num_of_epoch do:
    For prompt in Training_set do:
        response <- model.inference(policy, prompt)
        discriminator.train(Training_set.GPT_response, response)
        score = discriminator.inference(response)

        generation_loss = compute_generation_loss(generated_response, target_response)
        # or SFT loss if possible
        
        scoring_loss = torch.nn.functional.mse_loss(generated_score, target_score)
        
        total_loss = generation_loss + lambda * scoring_loss

        discriminator.update_and_backprop()
        model.update_and_backprop()

    end For
end For

# evaluation both base model and fine-tuned model
For model in [base_model, fine-tuned_model] do:
    For prompt in Test_set do:
        if use_RAG:
            rag_result <- search(prompt, Training_set)
            response <- model.inference_with_RAG(policy, prompt, rag_result)
        else:
            response <- model.inference(policy, prompt)
    end For
    response.evluate()
end For

In [8]:
%pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7383674a7f7aec344faa64897f371cc50b5a7ad505dd488c7a79cd24da42ffef
  Stored in directory: /home/uceehuf/.cache/pip/wheels/6f/d0/79/3beb1118c3c80946f4d13c0200a43cf56f35fc76ce6aac08ac
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [6]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

reference = [['this', 'is', 'a', 'test']]
candidate = ['this', 'is','a', 'test']

# Calculate BLEU score
smoothie = SmoothingFunction().method4
bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
print(f"BLEU score: {bleu_score:.4f}")


BLEU score: 1.0000


In [13]:
from rouge_score import rouge_scorer

reference = "This is a test."
candidate = " is test."

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference, candidate)
print(f"ROUGE-1: {scores['rouge1'].fmeasure:.4f}")
print(f"ROUGE-L: {scores['rougeL'].fmeasure:.4f}")


ROUGE-1: 0.6667
ROUGE-L: 0.6667


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

text = "This is a test sentence."
input_ids = tokenizer.encode(text, return_tensors='pt')

with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss)

print(f"Perplexity: {perplexity.item():.4f}")
